In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import forward
import os

BATCH_SIZE = 200
LEARNING_RATE_BASE = 0.1
LEARNING_RATE_DECAY = 0.99
REGULARIZER = 0.0001
STEPS = 50000
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH = "./model/"
MODEL_NAME = "mnist_model"


def backward(mnist):
    x = tf.placeholder(dtype=tf.float32,
    shape=[None, forward.INPUT_NODE])
    y_ = tf.placeholder(dtype=tf.float32,
    shape=[None, forward.OUTPUT_NODE])
    y = forward.forward(x,REGULARIZER)

    global_step = tf.Variable(0,trainable = False)

    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = y,
    labels= tf.argmax(y_,1))
    cem = tf.reduce_mean(ce)
    loss = cem + tf.add_n(tf.get_collection('losses'))

    learning_rate = tf.train.exponential_decay(
    LEARNING_RATE_BASE,
    global_step,
    mnist.train.num_examples/BATCH_SIZE,
    LEARNING_RATE_DECAY,
    staircase=True)

    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,
    global_step=global_step)

    ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    ema_op = ema.apply(tf.trainable_variables())
    with tf.control_dependencies([train_step,ema_op]):
        train_op = tf.no_op(name = 'train')

    saver = tf.train.Saver()

    with tf.Session() as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)

        for i  in range(STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _,loss_value,step = sess.run([train_op,loss,global_step],
            feed_dict={x:xs,y_:ys})
            if i % 1000 == 0:
                print("After %d training step(s)m loss on training batch is %g."%(step,loss_value))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH,MODEL_NAME),
                global_step=global_step)


def main():
    mnist = input_data.read_data_sets("./data/",one_hot = True)
    backward(mnist)

if __name__=='__main__':
    main()


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
After 1 training step(s)m loss on training batch is 2.80122.
After 1001 training step(s)m loss on training batch is 0.385062.
After 2001 training step(s)m loss on training batch is 0.254865.
After 3001 training step(s)m loss on training batch is

KeyboardInterrupt: 